École d'été GRAPHADON
=======
Travaux pratiques avec pyGSP
-----------

Nous allons voir une introduction pratique au traitement de signaux sur graphes (GSP - Graph Signal Processing). Nous utiliserons le package python [pyGSP](https://github.com/epfl-lts2/pygsp). Ce package, conçu par l'EPFL, permet d'effectuer une grande variété d'opérations pour manipuler des signaux sur graphes, en particulier pour le calcul et la manipulation de la Graph Fourier Transform (GFT) à des fins de filtrage par exemple. 

Vous devrez avoir installé au préalable ce package avec votre gestionnaire de package python préféré: `pip`ou `conda`. Avec `pip`faites la commande `python3 -m pip install pygsp`.

La documentation des fonctionnalités de pyGSP est acessible [ici](https://pygsp.readthedocs.io/en/v0.5.1/reference/index.html).

Commençons par importer les packages dont nous aurons besoin (si certains sont manquants, installez-les également).

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pygsp
